# Transformée de Fourier discrète (TFD)

In [8]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from ipywidgets import interactive, IntSlider, FloatSlider, Dropdown, VBox, HBox, Layout, Checkbox
from IPython.display import display
%matplotlib inline

# --- Constantes globales ---
T_AFFICHAGE = 5.0   # Durée d'affichage dans le domaine temporel (en secondes)

# --- Génération du signal discret ---
def generate_discrete_signal(signal_type, f0, Fe, N):
    """
    Génère un signal discret x[n] de longueur N échantillonné à Fe,
    pour un type de signal donné.
    
    signal_type : "Sinus", "Carré", "Triangle"
    f0          : fréquence fondamentale du signal
    Fe          : fréquence d'échantillonnage
    N           : nombre d'échantillons
    """
    n = np.arange(N)
    t = n / Fe

    if signal_type == "Sinus":
        x = np.sin(2 * np.pi * f0 * t)
    elif signal_type == "Carré":
        x = signal.square(2 * np.pi * f0 * t)
    elif signal_type == "Triangle":
        x = signal.sawtooth(2 * np.pi * f0 * t, width=0.5)
    else:
        x = np.sin(2 * np.pi * f0 * t)

    return t, x


# --- Fonction principale de tracé ---
def plot_fft_demo(signal_type, f0, Fe, N, K, show_fft_N, show_fft_K):
    """
    Démo interactive pour illustrer l'effet de :
    - N : nombre de points d'observation du signal
    - K : taille de la FFT (K >= N), avec zéro-padding si K > N.

    1er graphe (temps) : 5 s affichées
        * signal sur N points (bleu) si show_fft_N
        * signal sur K points (rouge), avec (K-N) zéros si K > N, si show_fft_K

    2e graphe (fréquence) : |FFT| tracée entre 0 et Fe
        * FFT sur N points (bleu) si show_fft_N
        * FFT sur K points (rouge) si show_fft_K

    3e graphe : zoom fréquentiel autour de f0
    """
    # Sécuriser les paramètres
    N = int(max(1, N))
    K = int(max(N, K))  # on force K >= N dans le calcul

    # --- Signal discret de longueur N ---
    t_n, x_n = generate_discrete_signal(signal_type, f0, Fe, N)

    # --- Zéro-padding pour obtenir K points ---
    x_k = np.zeros(K)
    x_k[:N] = x_n
    t_k = np.arange(K) / Fe

    # --- FFT sur N et sur K points ---
    X_N = np.fft.fft(x_n, n=N)
    freqs_N = np.arange(N) * Fe / N      # échantillons de 0 à (N-1)*Fe/N

    X_K = np.fft.fft(x_k, n=K)
    freqs_K = np.arange(K) * Fe / K      # échantillons de 0 à (K-1)*Fe/K

    # Magnitude normalisée (division par N pour garder la même échelle au pic)
    mag_N = np.abs(X_N) / N
    mag_K = np.abs(X_K) / N

    # --- Création des figures ---
    fig, (ax_time, ax_freq, ax_zoom) = plt.subplots(3, 1, figsize=(12, 11))

    # =========================
    #       DOMAINE TEMPS
    # =========================
    ax_time.set_title(
        f"Domaine temporel\n"
        f"Signal : {signal_type}, f0 = {f0:.1f} Hz, Fe = {Fe:.1f} Hz, "
        f"N = {N} points, K = {K} points (zéro-padding)"
    )

    # Masques pour rester dans les 5 s d'affichage
    mask_N_aff = t_n <= T_AFFICHAGE
    mask_K_aff = t_k <= T_AFFICHAGE

    # 1) Signal sur K points, avec zéros (ROUGE, tracé d'abord)
    if show_fft_K:
        ax_time.stem(
            t_k[mask_K_aff],
            x_k[mask_K_aff],
            basefmt=" ",
            linefmt='r-',
            markerfmt='r^',
            label=f"Signal zéro-paddé (K = {K})"
        )

    # 2) Signal sur N points (BLEU, au-dessus)
    if show_fft_N:
        ax_time.stem(
            t_n[mask_N_aff],
            x_n[mask_N_aff],
            basefmt=" ",
            linefmt='b-',
            markerfmt='bo',
            label=f"Signal discret (N = {N})"
        )

    ax_time.set_xlim(0, T_AFFICHAGE)
    ax_time.set_xlabel("Temps (s)")
    ax_time.set_ylabel("Amplitude")
    ax_time.grid(True)
    if show_fft_N or show_fft_K:
        ax_time.legend(loc="upper right")

    # =========================
    #     DOMAINE FRÉQUENCE
    # =========================
    ax_freq.set_title(
        "Domaine fréquentiel : magnitude de la FFT (|X[k]|)"
    )

    # 1) FFT sur K points (ROUGE, d'abord)
    if show_fft_K:
        ax_freq.stem(
            freqs_K,
            mag_K,
            basefmt=" ",
            linefmt='r-',
            markerfmt='r^',
            label="FFT sur K points (zéro-padding)"
        )

    # 2) FFT sur N points (BLEU, par dessus)
    if show_fft_N:
        ax_freq.stem(
            freqs_N,
            mag_N,
            basefmt=" ",
            linefmt='b-',
            markerfmt='bo',
            label="FFT sur N points"
        )

    ax_freq.set_xlim(0, Fe)
    ax_freq.set_xlabel("Fréquence (Hz)")
    ax_freq.set_ylabel("|X(f)| (normalisée)")
    ax_freq.grid(True)
    if show_fft_N or show_fft_K:
        ax_freq.legend(loc="upper right")

    # =========================
    #      ZOOM FRÉQUENTIEL
    # =========================
    ax_zoom.set_title("Zoom fréquentiel autour de f0")

    # Largeur de la fenêtre de zoom (en Hz)
    window_half = 5.0  # ±5 Hz autour de f0
    f_min_zoom = max(0.0, f0 - window_half)
    f_max_zoom = min(Fe, f0 + window_half)

    # Masques de zoom pour N et K
    mask_zoom_N = (freqs_N >= f_min_zoom) & (freqs_N <= f_max_zoom)
    mask_zoom_K = (freqs_K >= f_min_zoom) & (freqs_K <= f_max_zoom)

    # 1) FFT sur K points, zoomée (ROUGE d'abord)
    if show_fft_K and np.any(mask_zoom_K):
        ax_zoom.stem(
            freqs_K[mask_zoom_K],
            mag_K[mask_zoom_K],
            basefmt=" ",
            linefmt='r-',
            markerfmt='r^',
            label="FFT sur K points (zoom)"
        )

    # 2) FFT sur N points, zoomée (BLEU par dessus)
    if show_fft_N and np.any(mask_zoom_N):
        ax_zoom.stem(
            freqs_N[mask_zoom_N],
            mag_N[mask_zoom_N],
            basefmt=" ",
            linefmt='b-',
            markerfmt='bo',
            label="FFT sur N points (zoom)"
        )

    ax_zoom.set_xlim(f_min_zoom, f_max_zoom)
    ax_zoom.set_xlabel("Fréquence (Hz)")
    ax_zoom.set_ylabel("|X(f)| (normalisée)")
    ax_zoom.grid(True)
    if show_fft_N or show_fft_K:
        ax_zoom.legend(loc="upper right")

    plt.tight_layout()
    plt.show()


# --- Définition des widgets (larges, en 2 colonnes) ---
# Layouts
slider_layout = Layout(width='450px')  # largeur confortable pour les sliders
checkbox_layout = Layout(width='300px')
dropdown_layout = Layout(width='450px')
column_layout = Layout(width='50%')    # chaque colonne prend la moitié

signal_dropdown = Dropdown(
    options=["Sinus", "Carré", "Triangle"],
    value="Sinus",
    description="Signal :",
    layout=dropdown_layout,
    style={'description_width': '90px'}
)

f0_slider = FloatSlider(
    min=0.1, max=5.0, step=0.1, value=1.0,
    description="f0 (Hz) :",
    continuous_update=False,
    layout=slider_layout,
    style={'description_width': '90px'}
)

Fe_slider = FloatSlider(
    min=10.0, max=100.0, step=5.0, value=50.0,
    description="Fe (Hz) :",
    continuous_update=False,
    layout=slider_layout,
    style={'description_width': '90px'}
)

# N_max dépend de Fe : N_max = 5 * Fe
initial_N_max = int(T_AFFICHAGE * Fe_slider.value)

N_slider = IntSlider(
    min=4, max=initial_N_max, step=1, value=100,
    description="N (points) :",
    continuous_update=False,
    layout=slider_layout,
    style={'description_width': '110px'}
)

# K_max = 10 * N, initialement avec N_slider.value
initial_N = N_slider.value
initial_K_max = 10 * initial_N

K_slider = IntSlider(
    min=initial_N, max=initial_K_max, step=1, value=initial_N,
    description="K (FFT) :",
    continuous_update=False,
    layout=slider_layout,
    style={'description_width': '110px'}
)

fftN_checkbox = Checkbox(
    value=True,
    description="Signal et FFT (N points)",
    layout=checkbox_layout
)

fftK_checkbox = Checkbox(
    value=True,
    description="Signal et FFT (K points)",
    layout=checkbox_layout
)


# --- Logique dynamique ---
def update_N_bounds_on_Fe_change(change):
    """
    Quand Fe change :
      - on met à jour N_max = 5 * Fe
      - si N > N_max, on ramène N à N_max
      - on met à jour K en conséquence via enforce_K_from_N
    """
    new_Fe = change['new']
    N_max = int(T_AFFICHAGE * new_Fe)
    N_slider.max = N_max

    if N_slider.value > N_max:
        N_slider.value = N_max   # déclenchera enforce_K_from_N
    else:
        enforce_K_from_N(None)

def enforce_K_from_N(change):
    """
    À chaque changement de N :
      - K_min = N
      - K_max = 10 * N
      - K = N (K suit toujours N dès qu'on touche à N)
    """
    N_val = N_slider.value
    K_slider.min = N_val
    K_slider.max = 10 * N_val
    K_slider.value = N_val

def enforce_K_ge_N_on_K_change(change):
    """
    Si l'utilisateur essaie de mettre K < N ou K > 10N,
    on réimpose N <= K <= 10N.
    """
    new_K = change['new']
    N_val = N_slider.value
    if new_K < N_val:
        K_slider.value = N_val
    elif new_K > 10 * N_val:
        K_slider.value = 10 * N_val


Fe_slider.observe(update_N_bounds_on_Fe_change, names='value')
N_slider.observe(enforce_K_from_N, names='value')
K_slider.observe(enforce_K_ge_N_on_K_change, names='value')

# Initialisation cohérente
enforce_K_from_N(None)


# --- Construction de l'interface interactive ---
controls_left = VBox([
    signal_dropdown,
    f0_slider,
    Fe_slider,
], layout=column_layout)

controls_check = HBox([fftN_checkbox, fftK_checkbox])#, layout=Layout(width='100%'))

controls_right = VBox([
    N_slider,
    K_slider,
    controls_check,
    # fftN_checkbox,
    # fftK_checkbox
], layout=column_layout)



controls = HBox([controls_left, controls_right], layout=Layout(width='100%'))

interactive_plot = interactive(
    plot_fft_demo,
    signal_type=signal_dropdown,
    f0=f0_slider,
    Fe=Fe_slider,
    N=N_slider,
    K=K_slider,
    show_fft_N=fftN_checkbox,
    show_fft_K=fftK_checkbox
)

display(VBox([controls, interactive_plot.children[-1]]))

### Interprétation
On analyse en fréquence un signal discret sinusoidal, carré ou triangle (au choix) de fréquence $F_0$ et échantillonné à une fréquence $F_e$. On prélève $N$ échantillons de ce signal, qui sont tracés en bleu en fonction du temps. Ce signal de $N$ points est ensuite analysé en fréquence via la Transformée de Fourier Discrète (TFD) sur un nombre de point $K$, avec $K=N$ par défaut.

Le tracé au centre représente le module de cette TFD, représenté également en bleu pour des fréquences comprises entre $0$ et $F_e$. La TFD étant une grandeur discrète en fréquence et calculée sur $K$ points, on a donc $K$ échantillons représentés sur ce graphique. 

La figure du bas est seulement un zoom entre 0 et 6Hz permettant de mieux visualiser le comportement du spectre sur cette bande de fréquence où se situe la fréquence $F_0$.

**Questions** :
- peut-on mesurer exactement la fréquence $F_0$ du signal sur son spectre ?
- quel est l'effet d'une augmentation de la valeur de $N$ sur ce tracé ?

### Effet de $K$ : bourrage de zéros
On peut également augmenter la valeur de $K$ utilisée, avec $K\geq N$. Dans ce cas, le spectre est donc calculé sur un nombre de points possiblement supérieur au nombre de points $N$ du signal. En pratique, on peut montrer que calculer le spectre du signal de $N$ point sur un nombre de points en fréquence $K\geq N$ revient à calculer la TFD d'un nouveau signal constitué des $N$ points du signal de départ, enrichis de $K-N$ échantillons tous égaux à zéros : on vient de constituer un nouveau signal de $K$ points en y *ajoutant* des zéro. C'est une pratique qui s'appelle le **bourrage de zéros** ou **zero padding** en anglais.

Ce signal enrichis de 0 est représentés en rouge sur le graphique temporel, et son spectre est aussi tracé en rouge dans les 2 figures en fréquence.

**Questions** :
- quel est l'effet d'une augmentation de la valeur de $K$, à $N$ constant, sur le spectre du signal analysé ?

### Paramètres :
- Nature du signal : sinusoidal, carré ou triangle
- $F_0$ (Hz) : fréquence du sinus, ou fréquence fondamentale des signaux carré ou triangle
- $F_e$ (Hz) : fréquence d'échantillonnage
- $N$ : nombre de points du signal échantillonné
- $K$ : nombre de points en fréquence de la TFD du signal échantillonné
- 2 boites à cocher : permet d'afficher ou non le signal/spectre défini sur $N$ (signal original) ou $K$ (signal enrichi de zéros) points.
<br/><br/>
<br/><br/>
------
*[Sylvain ARGENTIERI](https://sylvain.argentieri.me)*
- *v1.0: version initiale (02/12/2025)*